In [453]:
"""
Скрипт для обработки данных  с портала трудвсем для БЦК
"""

'\nСкрипт для обработки данных  с портала трудвсем для БЦК\n'

In [454]:
import pandas as pd
import numpy as np
import openpyxl
import json
import ast
import re
import os
import datetime
import time
from dateutil import parser
pd.options.mode.chained_assignment = None


In [455]:
def extract_data_from_list_cell(cell:str,lst_need_keys:list):
    """
    Функция для извлечения данных из словаря в ячейке датафрейма
    """
    try:
        
        lst_lang = ast.literal_eval(cell) # превращаем в список
        if lst_lang:            
            out_str_lst = [] # создаем список содержащий выходные строки
            for lang_dict in lst_lang:
                lst_lang_str = [] # список для хранения значений извлеченных из словаря
                for idx,key in enumerate(lst_need_keys):
                    lst_lang_str.append(lang_dict.get(key,None))             
                    
                lst_lang_str = [value for value in lst_lang_str if value] # отбрасываем None
                single_lang_str = ','.join(lst_lang_str) # создаем строку для одного языка
                out_str_lst.append(single_lang_str)
            return ';'.join(out_str_lst)
                               
        else:
            return 'Не указано'
        

    except FileNotFoundError:
        return 'Не удалось обработать содержимое ячейки'

    

In [456]:
def clear_tag(cell):
    """
    Функция для очистки текста от тегов HTML
    """
    value = str(cell)
    if value != 'nan':        
        result = re.sub(r'<.*?>','',value)
        result = re.sub(r'&[a-z]*?;','',result)
        return result
    else:
        return None

def clear_bonus_tag_br(cell):
    """
    Функция для очистки данных в колонке Дополнительные бонусы
    """
    cell = str(cell)

    if cell !='nan':
        value = str(cell)
        result = re.sub(r'<.*?>','.',value)

        return result
    else:
        return None

In [457]:
def convert_date(cell):
    """
    Функция конвертации строки содержащей дату и время
    """
    value = str(cell)
    try:        
        if value !='nan':
            date_time = parser.parse(value).date() # извлекаем дату
            date_time = datetime.datetime.strftime(date_time,'%d.%m.%Y') # конфертируем в нужный формат
            return date_time

        else:
            return None
    except:
        return 'Не удалось обработать содержимое ячейки'
    

In [458]:
def prepare_data_vacancy(df:pd.DataFrame,dct_name_columns:dict,lst_columns:list)->pd.DataFrame:
    """
    Функция для обработки датафрейма с данными работы в России
    """
    df = df[dct_name_columns.keys()]
    df.rename(columns=dct_name_columns,inplace=True)
    # Обрабатываем обычные колонки
    
    df['Дополнительные бонусы'] = df['Дополнительные бонусы'].apply(clear_bonus_tag_br)
    df['Требования'] = df['Требования'].apply(clear_tag)
    df['Обязанности'] = df['Обязанности'].apply(clear_tag)
    
    # Числовые
    lst_number_columns = ['Требуемый опыт работы в годах','Минимальная зарплата','Максимальная зарплата','Количество рабочих мест']
    df[lst_number_columns] = df[lst_number_columns].fillna(0)
    df[lst_number_columns] = df[lst_number_columns].astype(int,errors='ignore')
  
    # Временные
    
    df['Дата размещения вакансии'] = df['Дата размещения вакансии'].apply(convert_date)
    df['Дата изменения вакансии'] = df['Дата изменения вакансии'].apply(convert_date)
    
    # Категориальные
    df['Квотируемое место'] = df['Квотируемое место'].apply(lambda x: 'Квотируемое место' if x == 'true' else None)
    df['Требуется медкнижка'] = df['Требуется медкнижка'].apply(lambda x:'Требуется медкнижка' if x == 'true' else None)
    df['Статус проверки вакансии'] = df['Статус проверки вакансии'].replace(dct_status_vacancy)
    # Начинаем извлекать данные из сложных колонок с json
    # данные по работодателю
    df['КПП работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('kpp','Не указано'))
    df['ОГРН работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('ogrn','Не указано'))
    df['Контактный телефон'] = df['Данные компании'].apply(lambda x: json.loads(x).get('phone','Не указано'))
    df['Email работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('email','Не указано'))
    df['Профиль работодателя'] = df['Данные компании'].apply(lambda x: json.loads(x).get('url','Не указано'))
    
    # Обрабатываем колонку с языками
    df['Требуемые языки'] = df['Данные по языкам'].apply(lambda x:extract_data_from_list_cell(x,['code_language','level']))
    df['Требуемые хардскиллы'] = df['Данные по хардскиллам'].apply(lambda x:extract_data_from_list_cell(x,['hard_skill_name']))
    df['Требуемые софтскиллы'] = df['Данные по софтскиллам'].apply(lambda x:extract_data_from_list_cell(x,['soft_skill_name']))
    
    df.drop(columns=['Данные компании','Данные по языкам','Данные по хардскиллам','Данные по софтскиллам'],inplace=True)
    
    df = df.reindex(columns=lst_columns)
    
    return df
    
    

In [459]:
region = 'Красноярский край'
file_data = 'data/vacancy.csv'
file_org = 'data/company.xlsx'
end_folder = 'data'

In [460]:
# колонки которые нужно оставить и переименовать
dct_name_columns = {'busy_type':'Тип занятости','contact_person':'Контактное лицо','date_create':'Дата размещения вакансии',
                    'date_modify':'Дата изменения вакансии','education':'Образование','education_speciality':'Требуемая специализация','is_quoted':'Квотируемое место',
                    'need_medcard':'Требуется медкнижка',
                   'other_vacancy_benefit':'Дополнительные бонусы','position_requirements':'Требования',
                    'position_responsibilities':'Обязанности','regionName':'Регион','required_experience':'Требуемый опыт работы в годах',
                    'retraining_capability':'Возможность переподготовки','required_certificates':'Требуемые доп. документы',
                   'required_drive_license':'Требуемые водительские права','retraining_grant':'Стипендия',
                    'retraining_grant_value':'Размер стипендии','salary':'Зарплата','salary_min':'Минимальная зарплата','salary_max':'Максимальная зарплата',
                    'schedule_type':'График работы','social_protected_ids':'Социально защищенная категория',
                   'source_type':'Источник вакансии','status':'Статус проверки вакансии','transport_compensation':'Компенсация транспорт',
                    'vacancy_address_additional_info':'Доп информация по адресу вакансии','vacancy_address':'Адрес вакансии',
                    'vacancy_address_latitude':'Долгота адрес вакансии','vacancy_address_longitude':'Широта адрес вакансии',
                   'vacancy_benefit_ids':'Бонусы','vacancy_name':'Вакансия','work_places':'Количество рабочих мест','professionalSphereName':'Сфера деятельности',
                    'full_company_name':'Полное название работодателя','company_inn':'ИНН работодателя','company':'Данные компании',
                    'languageKnowledge':'Данные по языкам','hardSkills':'Данные по хардскиллам','softSkills':'Данные по софтскиллам'}

In [461]:
dct_status_vacancy = {'ACCEPTED':'Данные вакансии проверены работодателем','AUTOMODERATION':'Автомодерация',
                      'REJECTED':'Отклонено','CHANGED':'Статус вакансии изменен',
                      'WAITING':'Ожидает подтверждения',}

In [462]:
t = time.localtime()  # получаем текущее время
current_time = time.strftime('%H_%M_%S', t)
current_date = time.strftime('%d_%m_%Y',t)

In [463]:
df = pd.read_csv(file_data,encoding='UTF-8',sep='|',dtype=str)


In [464]:
df.shape

(95676, 78)

In [465]:
company_df = pd.read_excel(file_org,dtype=str)

In [466]:
company_df

,Краткое название организации,ИНН
0,"АО ""Красноярский машиностроительный завод""",2462206345
1,"АО ""НПП ""Радиосвязь""",2460243408
2,"ООО ""Ачинский цемент""",2443030350
3,"ЗФ ПАО ""ГМК ""Норильский никель""",8401005730
4,"АО ""Информационные спутниковые системы имени М...",2452034898
5,"ООО ""Научно-производственное предприятие ""Авто...",2461217337
6,"АО ""Богучанский алюминиевый завод""",2465102746
7,"АО ""Испытательный технический центр - НПО ПМ""",2452027379
8,"ООО ""Вариант-999""",2466077228
9,"АО ""Русал Красноярский алюминиевый завод""",2465000141


In [467]:
lst_columns = ['Дата размещения вакансии','Дата изменения вакансии','Регион','Вакансия','Сфера деятельности','Количество рабочих мест',
               'Зарплата','Минимальная зарплата','Максимальная зарплата','График работы','Тип занятости','Образование','Требуемая специализация',
               'Требования','Обязанности','Бонусы','Дополнительные бонусы','Возможность переподготовки','Стипендия','Размер стипендии','Компенсация транспорт',
               'Квотируемое место','Социально защищенная категория',
               'Требуемый опыт работы в годах','Требуется медкнижка','Требуемые доп. документы','Требуемые водительские права',
               'Требуемые языки','Требуемые хардскиллы','Требуемые софтскиллы',
               'Источник вакансии','Статус проверки вакансии','Полное название работодателя','Адрес вакансии','Доп информация по адресу вакансии',
               'ИНН работодателя','КПП работодателя','ОГРН работодателя','Контактное лицо','Контактный телефон','Email работодателя',
               'Профиль работодателя','Долгота адрес вакансии','Широта адрес вакансии']

In [468]:
df = df[df['regionName'] == region]

In [469]:
df.shape

(17884, 78)

In [470]:
all_status_prepared_df = prepare_data_vacancy(df,dct_name_columns,lst_columns) # получаем обработанный датафрейм со всеми статусами вакансий



In [471]:
all_status_prepared_df.shape

(17884, 44)

In [472]:
prepared_df = all_status_prepared_df[all_status_prepared_df['Статус проверки вакансии'] == 'Данные вакансии проверены работодателем']
union_company_df_columns = list(prepared_df.columns).insert(0,'Организация')
union_company_df = pd.DataFrame(columns=union_company_df_columns)

In [473]:
prepared_df.shape

(17811, 44)

In [474]:
dct_df = dict() # словарь для хранения данных для каждого инн
used_name_sheet = set() # множество для хранения использованных названий листов

In [475]:
org_folder = f'{end_folder}/Вакансии по организациям/{current_date}' # создаем папку куда будем складывать вакансии по организациям
if not os.path.exists(org_folder):
    os.makedirs(org_folder)


# Собираем датафреймы по ИНН
for idx,row in enumerate(company_df.itertuples()):
    name_company = row[1] # название компании
    
    inn_company = row[2] # инн компании
    temp_df = prepared_df[prepared_df['ИНН работодателя'] ==inn_company] # фильтруем по инн
    temp_df.sort_values(by=['Вакансия'],inplace=True) # сортируем
    name_company = re.sub(r'[\r\b\n\t<>:"?*|\\/]', '_', name_company)# очищаем от лишних символов

    temp_df.to_excel(f'{org_folder}/{name_company}.xlsx',index=False) # сохраняем
    # создаем отдельный файл в котором будут все вакансии по выбранным компаниям
    temp_df.insert(0,'Организация',name_company)
    union_company_df = pd.concat([union_company_df,temp_df],ignore_index=True)
    
    
    

In [476]:
union_company_df

,Организация,Дата размещения вакансии,Дата изменения вакансии,Регион,Вакансия,Сфера деятельности,Количество рабочих мест,Зарплата,Минимальная зарплата,Максимальная зарплата,...,Доп информация по адресу вакансии,ИНН работодателя,КПП работодателя,ОГРН работодателя,Контактное лицо,Контактный телефон,Email работодателя,Профиль работодателя,Долгота адрес вакансии,Широта адрес вакансии
0,АО _Красноярский машиностроительный завод_,28.02.2024,04.03.2024,Красноярский край,Архивариус,Машиностроение,1,от 33000,33000,37000,...,"Остановка ""ДК 1 Мая"", бело-синее здание, Отдел...",2462206345,246201001,1082468060553,Екатерина Дмитриевна Смоленцева,+7(391)2644712,info@krskmz.ru,https://trudvsem.ru/company/1082468060553,56.014841,92.983321
1,АО _Красноярский машиностроительный завод_,16.11.2023,05.03.2024,Красноярский край,Ведущий специалист по подготовке производства ...,Машиностроение,1,от 50000,50000,57600,...,"Остановка ""ДК 1 Мая"", бело-синее здание, Отдел...",2462206345,246201001,1082468060553,Екатерина Дмитриевна Смоленцева,+7(391)2644712,info@krskmz.ru,https://trudvsem.ru/company/1082468060553,56.015475,92.986331
2,АО _Красноярский машиностроительный завод_,12.05.2022,05.03.2024,Красноярский край,Водитель автомобиля,Машиностроение,3,от 40000,40000,49900,...,"Остановка ""ДК 1 Мая"", бело-синее здание, Отдел...",2462206345,246201001,1082468060553,Смоленцева Екатерина,+7(391)2644712,info@krskmz.ru,https://trudvsem.ru/company/1082468060553,56.015475,92.986331
3,АО _Красноярский машиностроительный завод_,28.04.2022,05.03.2024,Красноярский край,Водитель электро- и автотележки-грузчик,Машиностроение,1,от 30800,30800,35000,...,"Остановка ""ДК 1 Мая"", бело-синее здание, Отдел...",2462206345,246201001,1082468060553,Смоленцева Екатерина,+7(391)2644712,info@krskmz.ru,https://trudvsem.ru/company/1082468060553,56.015475,92.986331
4,АО _Красноярский машиностроительный завод_,05.04.2022,21.02.2024,Красноярский край,Горничная,"Услуги населению, сервисное обслуживание",2,от 32000,32000,0,...,NaN,2462206345,246243002,1082468060553,Соколова Ирина Леонидовна,+7(391)2644712,info@krskmz.ru,https://trudvsem.ru/company/a84f4330-a664-11ec...,55.949229,92.842502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,ФГУП _Горно-химический комбинат_,09.02.2024,12.03.2024,Красноярский край,инженер-технолог (химик),Производство,1,от 60000,60000,0,...,NaN,2452000401,245201001,1022401404871,Рябова Елена Владимировна,+7(391)2662337,sibghk@rosatom.ru,https://trudvsem.ru/company/1022401404871,56.260483,93.537232
1045,ФГУП _Горно-химический комбинат_,09.02.2024,12.03.2024,Красноярский край,инженер-технолог (химик),Производство,1,от 60000,60000,0,...,NaN,2452000401,245201001,1022401404871,Рябова Елена Владимировна,+7(391)2662337,sibghk@rosatom.ru,https://trudvsem.ru/company/1022401404871,56.260483,93.537232
1046,ФГУП _Горно-химический комбинат_,12.02.2024,12.02.2024,Красноярский край,инженер-технолог (химик),Производство,1,от 60000,60000,0,...,NaN,2452000401,245201001,1022401404871,Рябова Елена Владимировна,+7(391)2662337,sibghk@rosatom.ru,https://trudvsem.ru/company/1022401404871,56.260483,93.537232
1047,ФГУП _Горно-химический комбинат_,16.01.2024,12.03.2024,Красноярский край,инженер-технолог (химик),Производство,6,от 60000,60000,0,...,NaN,2452000401,245201001,1022401404871,Рябова Елена Владимировна,+7(391)2662337,sibghk@rosatom.ru,https://trudvsem.ru/company/1022401404871,56.260483,93.537232


In [477]:
union_company_df.to_excel(f'{org_folder}/Общий файл.xlsx',index=False)

In [478]:
with pd.ExcelWriter(f'{end_folder}/Данные по региону от {current_time}.xlsx') as writer:
    prepared_df.to_excel(writer,sheet_name='Только подтвержденные вакансии',index=0)
    df.to_excel(writer,sheet_name='Необработанные данные',index=1)
    all_status_prepared_df.to_excel(writer,sheet_name='Вакансии со всеми статусами',index=2)

In [479]:
# Свод по региону 


In [480]:
# Свод по количеству рабочих мест по отраслям

svod_vac_reg_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]})
svod_vac_reg_region_df = svod_vac_reg_region_df.droplevel(level=0,axis=1) # убираем мультииндекс


svod_vac_reg_region_df.sort_values(by=['sum'],ascending=False,inplace=True)
svod_vac_reg_region_df.loc['Итого'] = svod_vac_reg_region_df['sum'].sum()
svod_vac_reg_region_df.rename(columns={'sum':'Количество вакансий'},inplace=True)
svod_vac_reg_region_df=svod_vac_reg_region_df.reset_index()

In [481]:
# Свод по количеству рабочих мест по организациям
svod_vac_org_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]})
svod_vac_org_region_df = svod_vac_org_region_df.droplevel(level=0,axis=1) # убираем мультииндекс
svod_vac_org_region_df.sort_values(by=['sum'],ascending=False,inplace=True)
svod_vac_org_region_df.loc['Итого'] = svod_vac_org_region_df['sum'].sum()
svod_vac_org_region_df.rename(columns={'sum':'Количество вакансий'},inplace=True)
svod_vac_org_region_df=svod_vac_org_region_df.reset_index()

In [482]:
# Свод по средней и медианной минимальной зарплате для сфер деятельности
svod_shpere_pay_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности'],
              values = ['Минимальная зарплата'],
              aggfunc={'Минимальная зарплата':[np.mean,np.median]},
                                        fill_value=0)
svod_shpere_pay_region_df = svod_shpere_pay_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_pay_region_df = svod_shpere_pay_region_df.astype(int,errors='ignore')
svod_shpere_pay_region_df.columns = ['Средняя ариф. минимальная зп','Медианная минимальная зп']
svod_shpere_pay_region_df = svod_shpere_pay_region_df.reset_index()




In [483]:
# Свод по средней и медианной минимальной зарплате для работодателей
svod_org_pay_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Сфера деятельности'],
              values = ['Минимальная зарплата'],
              aggfunc={'Минимальная зарплата':[np.mean,np.median]},
                                        fill_value=0)
svod_org_pay_region_df = svod_org_pay_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_pay_region_df = svod_org_pay_region_df.astype(int,errors='ignore')
svod_org_pay_region_df.columns = ['Средняя ариф. минимальная зп','Медианная минимальная зп']
svod_org_pay_region_df = svod_org_pay_region_df.reset_index()


In [484]:
# Свод по требуемому образованию для сфер деятельности
svod_shpere_educ_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Образование'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_educ_region_df = svod_shpere_educ_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_educ_region_df = svod_shpere_educ_region_df.astype(int,errors='ignore')
svod_shpere_educ_region_df.columns = ['Количество вакансий']
svod_shpere_educ_region_df = svod_shpere_educ_region_df.reset_index()

In [485]:
# Свод по требуемому образованию для работодателей
svod_org_educ_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Образование'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_educ_region_df = svod_org_educ_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_educ_region_df = svod_org_educ_region_df.astype(int,errors='ignore')
svod_org_educ_region_df.columns = ['Количество вакансий']
svod_org_educ_region_df = svod_org_educ_region_df.reset_index()

In [486]:
# Свод по графику работы для сфер деятельности
svod_shpere_schedule_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','График работы'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_schedule_region_df = svod_shpere_schedule_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_schedule_region_df = svod_shpere_schedule_region_df.astype(int,errors='ignore')
svod_shpere_schedule_region_df.columns = ['Количество вакансий']
svod_shpere_schedule_region_df = svod_shpere_schedule_region_df.reset_index()

In [487]:
# Свод по графику работы для работодателей
svod_org_schedule_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','График работы'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_schedule_region_df = svod_org_schedule_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_schedule_region_df = svod_org_schedule_region_df.astype(int,errors='ignore')
svod_org_schedule_region_df.columns = ['Количество вакансий']
svod_org_schedule_region_df = svod_org_schedule_region_df.reset_index()

In [488]:
# Свод по типу занятости для сфер деятельности
svod_shpere_type_job_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Тип занятости'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_type_job_region_df = svod_shpere_type_job_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_type_job_region_df = svod_shpere_type_job_region_df.astype(int,errors='ignore')
svod_shpere_type_job_region_df.columns = ['Количество вакансий']
svod_shpere_type_job_region_df = svod_shpere_type_job_region_df.reset_index()

In [489]:
# Свод по типу занятости для работодателей
svod_org_type_job_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Тип занятости'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_type_job_region_df = svod_org_type_job_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_type_job_region_df = svod_org_type_job_region_df.astype(int,errors='ignore')
svod_org_type_job_region_df.columns = ['Количество вакансий']
svod_org_type_job_region_df = svod_org_type_job_region_df.reset_index()


In [490]:
svod_org_type_job_region_df

,Полное название работодателя,Тип занятости,Количество вакансий
0,"""АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК"" (АКЦИОНЕРНОЕ ОБЩ...",Полная занятость,3
1,"""ВОСТОЧНЫЙ ФИЛИАЛ"" - ФИЛИАЛ КРАЕВОГО ГОСУДАРСТ...",Полная занятость,6
2,"""КДЛ КРАСНОЯРСК"" ООО ""КДЛ СИБИРЬ""",Полная занятость,1
3,"""КРАСНОЯРСКИЙ""",Полная занятость,10
4,"""МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ""МОЛОДЕЖНЫ...",Временная,4
...,...,...,...
3751,муниципальное дошкольное образовательное бюдже...,Временная,2
3752,муниципальное казенное общеобразовательное учр...,Частичная занятость,1
3753,муниципальное общеобразовательное бюджетное у...,Временная,2
3754,муниципальное общеобразовательное бюджетное у...,Полная занятость,5


In [491]:
# Свод по квотируемым местам для сфер деятельности
svod_shpere_quote_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Квотируемое место'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_quote_region_df = svod_shpere_quote_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_quote_region_df = svod_shpere_quote_region_df.astype(int,errors='ignore')
svod_shpere_quote_region_df.columns = ['Количество вакансий']
svod_shpere_quote_region_df = svod_shpere_quote_region_df.reset_index()

In [492]:
# Свод по квотируемым местам для работодателей
svod_org_quote_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Квотируемое место'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_quote_region_df = svod_org_quote_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_quote_region_df = svod_org_quote_region_df.astype(int,errors='ignore')
svod_org_quote_region_df.columns = ['Количество вакансий']
svod_org_quote_region_df = svod_org_quote_region_df.reset_index()

In [495]:
# Свод по требуемому опыту для сфер деятельности
svod_shpere_exp_region_df=pd.pivot_table(prepared_df,
              index=['Сфера деятельности','Требуемый опыт работы в годах'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_shpere_exp_region_df = svod_shpere_exp_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_shpere_exp_region_df = svod_shpere_exp_region_df.astype(int,errors='ignore')
svod_shpere_exp_region_df.columns = ['Количество вакансий']
svod_shpere_exp_region_df = svod_shpere_exp_region_df.reset_index()

In [498]:
# Свод по требуемому опыту для работодателей
svod_org_exp_region_df=pd.pivot_table(prepared_df,
              index=['Полное название работодателя','Требуемый опыт работы в годах'],
              values = ['Количество рабочих мест'],
              aggfunc={'Количество рабочих мест':[np.sum]},
                                        fill_value=0)
svod_org_exp_region_df = svod_org_exp_region_df.droplevel(level=0,axis=1) # убираем мультииндекс

svod_org_exp_region_df = svod_org_exp_region_df.astype(int,errors='ignore')
svod_org_exp_region_df.columns = ['Количество вакансий']
svod_org_exp_region_df = svod_org_exp_region_df.reset_index()

In [499]:
svod_org_exp_region_df

,Полное название работодателя,Требуемый опыт работы в годах,Количество вакансий
0,"""АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК"" (АКЦИОНЕРНОЕ ОБЩ...",1,3
1,"""ВОСТОЧНЫЙ ФИЛИАЛ"" - ФИЛИАЛ КРАЕВОГО ГОСУДАРСТ...",0,5
2,"""ВОСТОЧНЫЙ ФИЛИАЛ"" - ФИЛИАЛ КРАЕВОГО ГОСУДАРСТ...",3,1
3,"""КДЛ КРАСНОЯРСК"" ООО ""КДЛ СИБИРЬ""",1,1
4,"""КРАСНОЯРСКИЙ""",0,9
...,...,...,...
4891,муниципальное казенное общеобразовательное учр...,0,1
4892,муниципальное общеобразовательное бюджетное у...,0,4
4893,муниципальное общеобразовательное бюджетное у...,3,2
4894,муниципальное общеобразовательное бюджетное у...,5,1


In [494]:
print('Lindy Booth!')

Lindy Booth!
